In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os

from keras.models import Model
from keras.layers import Conv2D, Input, Activation, BatchNormalization, Add, UpSampling2D, ZeroPadding2D
from keras.utils import get_file
import numpy as np

def normalize_image(image):
    """Normalize the image for the Hourglass network.
    # Arguments
      image: BGR uint8
    # Returns
      float32 image with the same shape as the input
    """
    mean = [0.40789655, 0.44719303, 0.47026116]
    std = [0.2886383, 0.27408165, 0.27809834]
    return ((np.float32(image) / 255.) - mean) / std


def HourglassNetwork(heads, num_stacks, cnv_dim=256, dims=[256, 384, 384, 384, 512]):
    """Instantiates the Hourglass architecture.
    Optionally loads weights pre-trained on COCO.
    Note that the data format convention used by the model is
    the one specified in your Keras config at `~/.keras/keras.json`.
    # Arguments
        num_stacks: number of hourglass modules.
        cnv_dim: number of filters after the resolution is decreased.
        inres: network input shape, should be a multiple of 128.
        weights: one of `None` (random initialization),
              'ctdet_coco' (pre-training on COCO for 2D object detection),
              'hpdet_coco' (pre-training on COCO for human pose detection),
              or the path to the weights file to be loaded.
        dims: numbers of channels in the hourglass blocks.
    # Returns
        A Keras model instance.
    # Raises
        ValueError: in case of invalid argument for `weights`,
            or invalid input shape.
    """
    input_layer = Input(shape=(512, 512, 1), name='HGInput')
    inter = pre(input_layer, cnv_dim)
    prev_inter = None
    outputs = []
    for i in range(num_stacks):
        prev_inter = inter
        _heads, inter = hourglass_module(heads, inter, cnv_dim, i, dims)
        outputs.extend(_heads)
        if i < num_stacks - 1:
            inter_ = Conv2D(cnv_dim, 1, use_bias=False, name='inter_.%d.0' % i)(prev_inter)
            inter_ = BatchNormalization(epsilon=1e-5, name='inter_.%d.1' % i)(inter_)
    
            cnv_ = Conv2D(cnv_dim, 1, use_bias=False, name='cnv_.%d.0' % i)(inter)
            cnv_ = BatchNormalization(epsilon=1e-5, name='cnv_.%d.1' % i)(cnv_)
    
            inter = Add(name='inters.%d.inters.add' % i)([inter_, cnv_])
            inter = Activation('relu', name='inters.%d.inters.relu' % i)(inter)
            inter = residual(inter, cnv_dim, 'inters.%d' % i)
    
    
    flattened_output = Flatten()(outputs[-1])
    dense = Dense(128, activation='relu')(flattened_output)
    chest_out = Dense(1, activation='linear', name='chest_out')(dense)
    waist_out = Dense(1, activation='linear', name='waist_out')(dense)
    pelvis_out = Dense(1, activation='linear', name='pelvis_out')(dense)
    bicep_out = Dense(1, activation='linear', name='bicep_out')(dense)
    thigh_out = Dense(1, activation='linear', name='thigh_out')(dense)
    shoulder_to_wrist_out = Dense(1, activation='linear', name='shoulder_to_wrist_out')(dense)
    leg_out = Dense(1, activation='linear', name='leg_out')(dense)
    calf_out = Dense(1, activation='linear', name='calf_out')(dense)
    wrist_out = Dense(1, activation='linear', name='wrist_out')(dense)
    shoulder_to_shoulder_out = Dense(1, activation='linear', name='shoulder_to_shoulder_out')(dense)

    model = Model(inputs=input_layer, outputs=[
        chest_out,
        waist_out,
        pelvis_out,
        bicep_out,
        thigh_out,
        shoulder_to_wrist_out,
        leg_out,
        calf_out,
        wrist_out,
        shoulder_to_shoulder_out,
    ])

    return model


def hourglass_module(heads, bottom, cnv_dim, hgid, dims):
    # create left features , f1, f2, f4, f8, f16 and f32
    lfs = left_features(bottom, hgid, dims)

    # create right features, connect with left features
    rf1 = right_features(lfs, hgid, dims)
    rf1 = convolution(rf1, 3, cnv_dim, name='cnvs.%d' % hgid)

    # add 1x1 conv with two heads, inter is sent to next stage
    # head_parts is used for intermediate supervision
    heads = create_heads(heads, rf1, hgid)
    return heads, rf1


def convolution(_x, k, out_dim, name, stride=1):
    padding = (k - 1) // 2
    _x = ZeroPadding2D(padding=padding, name=name + '.pad')(_x)
    _x = Conv2D(out_dim, k, strides=stride, use_bias=False, name=name + '.conv')(_x)
    _x = BatchNormalization(epsilon=1e-5, name=name + '.bn')(_x)
    _x = Activation('relu', name=name + '.relu')(_x)
    return _x


def residual(_x, out_dim, name, stride=1):
    shortcut = _x
    num_channels = _x.shape[-1]
    _x = ZeroPadding2D(padding=1, name=name + '.pad1')(_x)
    _x = Conv2D(out_dim, 3, strides=stride, use_bias=False, name=name + '.conv1')(_x)
    _x = BatchNormalization(epsilon=1e-5, name=name + '.bn1')(_x)
    _x = Activation('relu', name=name + '.relu1')(_x)

    _x = Conv2D(out_dim, 3, padding='same', use_bias=False, name=name + '.conv2')(_x)
    _x = BatchNormalization(epsilon=1e-5, name=name + '.bn2')(_x)

    if num_channels != out_dim or stride != 1:
        shortcut = Conv2D(out_dim, 1, strides=stride, use_bias=False, name=name + '.shortcut.0')(
            shortcut)
        shortcut = BatchNormalization(epsilon=1e-5, name=name + '.shortcut.1')(shortcut)

    _x = Add(name=name + '.add')([_x, shortcut])
    _x = Activation('relu', name=name + '.relu')(_x)
    return _x


def pre(_x, num_channels):
    # front module, input to 1/4 resolution
    _x = convolution(_x, 7, 128, name='pre.0', stride=2)
    _x = residual(_x, num_channels, name='pre.1', stride=2)
    return _x


def left_features(bottom, hgid, dims):
    # create left half blocks for hourglass module
    # f1, f2, f4 , f8, f16, f32 : 1, 1/2, 1/4 1/8, 1/16, 1/32 resolution
    # 5 times reduce/increase: (256, 384, 384, 384, 512)
    features = [bottom]
    for kk, nh in enumerate(dims):
        pow_str = ''
        for _ in range(kk):
            pow_str += '.center'
        _x = residual(features[-1], nh, name='kps.%d%s.down.0' % (hgid, pow_str), stride=2)
        _x = residual(_x, nh, name='kps.%d%s.down.1' % (hgid, pow_str))
        features.append(_x)
    return features


def connect_left_right(left, right, num_channels, num_channels_next, name):
    # left: 2 residual modules
    left = residual(left, num_channels_next, name=name + 'skip.0')
    left = residual(left, num_channels_next, name=name + 'skip.1')

    # up: 2 times residual & nearest neighbour
    out = residual(right, num_channels, name=name + 'out.0')
    out = residual(out, num_channels_next, name=name + 'out.1')
    out = UpSampling2D(name=name + 'out.upsampleNN')(out)
    out = Add(name=name + 'out.add')([left, out])
    return out


def bottleneck_layer(_x, num_channels, hgid):
    # 4 residual blocks with 512 channels in the middle
    pow_str = 'center.' * 5
    _x = residual(_x, num_channels, name='kps.%d.%s0' % (hgid, pow_str))
    _x = residual(_x, num_channels, name='kps.%d.%s1' % (hgid, pow_str))
    _x = residual(_x, num_channels, name='kps.%d.%s2' % (hgid, pow_str))
    _x = residual(_x, num_channels, name='kps.%d.%s3' % (hgid, pow_str))
    return _x


def right_features(leftfeatures, hgid, dims):
    rf = bottleneck_layer(leftfeatures[-1], dims[-1], hgid)
    for kk in reversed(range(len(dims))):
        pow_str = ''
        for _ in range(kk):
            pow_str += 'center.'
        rf = connect_left_right(leftfeatures[kk], rf, dims[kk], dims[max(kk - 1, 0)], name='kps.%d.%s' % (hgid, pow_str))
    return rf


def create_heads(heads, rf1, hgid):
    _heads = []
    for head in sorted(heads):
        num_channels = heads[head]
        _x = Conv2D(256, 3, use_bias=True, padding='same', name=head + '.%d.0.conv' % hgid)(rf1)
        _x = Activation('relu', name=head + '.%d.0.relu' % hgid)(_x)
        _x = Conv2D(num_channels, 1, use_bias=True, name=head + '.%d.1' % hgid)(_x)
        _heads.append(_x)
    return _heads


In [2]:
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
import os
import random

import cv2
import keras.utils
import numpy as np
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Model

keras.utils.set_random_seed(42)
np.random.seed(42)
random.seed(42)


heads = {'regression': 10}
model = HourglassNetwork(heads, num_stacks=2)

###TRAINING VARIABLES

sample_count_surreact = 100
sample_count_bodym = 100
epoch_count = 100
batch_size = 1
validation_count = 20
initial_learning_rate=.001
decay_steps= (sample_count_surreact + sample_count_bodym)/batch_size
decay_rate=0.97
model_name="CenterNet"

folder_path = f"{model_name} - SR{sample_count_surreact+sample_count_bodym}s{epoch_count}e{validation_count}v {"" if initial_learning_rate == 0.001 else initial_learning_rate + "ilr"}{decay_steps}s{decay_rate}d"

try:
    os.mkdir(os.path.join("./logs/" + folder_path))
except OSError as error:
    print(error)
    

#model.save_weights("default.weights.h5")
# 
# model.summary()
# 
# from keras.utils import plot_model
# 
# plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True, show_layer_activations=True)

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


2024-05-31 23:55:04.734777: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-31 23:55:06.135567: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


[Errno 17] File exists: './logs/CenterNet - SR200s100e20v 200.0s0.97d'


In [3]:
import numpy as np
from keras.utils import Sequence
import os
import cv2


class DatasetLoader(Sequence):

    def __init__(self, image_location, data_location, samples, batch_size, dataset, shuffle=True, seed=0,
                 input_dimensions=(240, 320), prefix="Avatar_", side=False, height=False):
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.seed = seed
        self.input_dimensions = input_dimensions
        self.prefix = prefix
        self.side = side
        self.height = height

        super().__init__(workers=5, use_multiprocessing=True)
        self.IDs = []
        self._load_data(samples, data_location, image_location, dataset)
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(len(self.IDs) / self.batch_size))

    def _load_data(self, samples, data_locations, image_locations, datasets):
        self.IDs = []
        self.data = dict()
        id_counter = 0
        for data_source_index in range(len(image_locations)):
            source_image_count = samples[data_source_index]
            source_data_location = data_locations[data_source_index]
            source_image_location = image_locations[data_source_index]
            source_dataset = datasets[data_source_index]

            for index in range(source_image_count):
                self.IDs.append(id_counter)
                self.data[id_counter] = [source_image_location,
                                         source_dataset,
                                         index,
                                         np.load(os.path.join(source_data_location + self.prefix + f"{index:06d}.npy"))]
                id_counter += 1

    def __getitem__(self, index):
        X = np.empty(shape=(self.batch_size, 512, 512))
        if self.side:
            X = np.empty(shape=(self.batch_size, 512, 512, 2))
        y = {
            'chest_out': [],
            'waist_out': [],
            'pelvis_out': [],
            'bicep_out': [],
            'thigh_out': [],
            'shoulder_to_wrist_out': [],
            'leg_out': [],
            'calf_out': [],
            'wrist_out': [],
            'shoulder_to_shoulder_out': [],
        }

        height = []
        start_index = index * self.batch_size + 1
        for i in range(self.batch_size):
            sample_ID = self.IDs[(start_index + i)]
            sample_properties = self.data[sample_ID]
            current_measurement = sample_properties[3]
            current_filename = f"{self.prefix}{sample_properties[2]:06d}.png"


            if self.side:
                X[i, :, :, 0] = cv2.imread(os.path.join(sample_properties[0] + "images_front/" + current_filename), cv2.IMREAD_GRAYSCALE)
                X[i, :, :, 1] = cv2.imread(os.path.join(sample_properties[0] + "images_side/" + current_filename), cv2.IMREAD_GRAYSCALE)
            else:
                X[i,] = cv2.imread(os.path.join(sample_properties[0]+ "images_front/" + current_filename), cv2.IMREAD_GRAYSCALE)

            if sample_properties[1] == "Surreact":
                y['chest_out'].append([current_measurement[0]])
                y['waist_out'].append([current_measurement[1]])
                y['pelvis_out'].append([current_measurement[2]])
                y['bicep_out'].append([current_measurement[4]])
                y['thigh_out'].append([current_measurement[5]])
                y['shoulder_to_wrist_out'].append([current_measurement[7]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[9]])
                y['wrist_out'].append([current_measurement[11]])
                y['shoulder_to_shoulder_out'].append([current_measurement[13]])
                if self.height:
                    height.append(current_measurement[16])

            if sample_properties[1] == "BodyM":
                y['chest_out'].append([current_measurement[4]])
                y['waist_out'].append([current_measurement[12]])
                y['pelvis_out'].append([current_measurement[7]])
                y['bicep_out'].append([current_measurement[2]])
                y['thigh_out'].append([current_measurement[11]])
                y['shoulder_to_wrist_out'].append([current_measurement[1]])
                y['leg_out'].append([current_measurement[8]])
                y['calf_out'].append([current_measurement[3]])
                y['wrist_out'].append([current_measurement[13]])
                y['shoulder_to_shoulder_out'].append([current_measurement[9]])
                if self.height:
                    height.append(current_measurement[6])

        for key, value in y.items():
            y[key] = np.array(value)

        if self.height:
            height = np.array(height)
            return [X, height], y

        return X, y

    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.IDs)
        else:
            self.IDs = np.arange(len(self.IDs))


In [4]:
quickTrain = {
                'image_location': ["Export/Surreact-APoseCenterNet/train/"], 
                'data_location': ["Export/Surreact-APose/train/measurements/"], 
                'samples': [sample_count_surreact, sample_count_bodym], 
                'dataset': ["Surreact"],
                'batch_size': batch_size, 
                'seed':  69,#np.random.randint(0, 10000), 
                'shuffle': True}

quickValidate = {
                'image_location': ["Export/Surreact-APoseCenterNet/test/"],
                'data_location': ["Export/Surreact-APose/test/measurements/"],
                'samples': [validation_count],
                'batch_size': batch_size,
                'dataset': ["Surreact"],
                'seed': 69, #np.random.randint(0, 10000),
                }

train_generator = DatasetLoader(**quickTrain)
validation_generator = DatasetLoader(**quickValidate)

In [5]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras.losses import MeanSquaredError

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=decay_steps,
    decay_rate=decay_rate,
    staircase=True)

stop_no_improvement = keras.callbacks.EarlyStopping(monitor="loss", patience=3)

model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss={
        'chest_out': MeanSquaredError(),
        'waist_out': MeanSquaredError(),
        'pelvis_out': MeanSquaredError(),
        'bicep_out': MeanSquaredError(),
        'thigh_out': MeanSquaredError(),
        'shoulder_to_wrist_out': MeanSquaredError(),
        'leg_out': MeanSquaredError(),
        'calf_out': MeanSquaredError(),
        'wrist_out': MeanSquaredError(),
        'shoulder_to_shoulder_out': MeanSquaredError(),
    },
    metrics={
        'chest_out': ['mae'],
        'waist_out': ['mae'],
        'pelvis_out': ['mae'],
        'bicep_out': ['mae'],
        'thigh_out': ['mae'],
        'shoulder_to_wrist_out': ['mae'],
        'leg_out': ['mae'],
        'calf_out': ['mae'],
        'wrist_out': ['mae'],
        'shoulder_to_shoulder_out': ['mae'],
    },
)

checkpoint_filepath = f'./models/{model_name}.checkpoint.model.keras'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True)

model.fit(
    x=train_generator,
    validation_data=validation_generator,
    callbacks=[

        TensorBoard(write_graph=True, log_dir="./logs/" + folder_path),
        model_checkpoint_callback,
        stop_no_improvement
    ],
    batch_size=batch_size,
    epochs=epoch_count,
    verbose=1
)

model.save(f"./models/full/{model_name}.keras") 

RuntimeError: Unable to automatically build the model. Please build it yourself before calling fit/evaluate/predict. A model is 'built' when its variables have been created and its `self.built` attribute is True. Usually, calling the model on a batch of data is the right way to build it.
Exception encountered:
'Exception encountered when calling Conv2D.call().

[1mCUDA out of memory. Tried to allocate 20.00 MiB. GPU 0 has a total capacity of 5.77 GiB of which 148.62 MiB is free. Process 27029 has 2.36 GiB memory in use. Including non-PyTorch memory, this process has 2.30 GiB memory in use. Of the allocated memory 2.16 GiB is allocated by PyTorch, and 8.71 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)[0m

Arguments received by Conv2D.call():
  • in